## Environment Setup and Package Installation

This cell handles the initial setup by installing all the necessary libraries needed for training the models. I created a helper function called `pip_install()` that wraps the subprocess command to install packages quietly, keeping the output clean. The main packages being installed include transformers for working with BERT models, datasets for managing the training data, accelerate to speed up the training process, evaluate for computing metrics, and openpyxl for exporting results to Excel later.

After installation, the cell checks the environment to confirm everything is ready. It imports torch and checks whether a GPU is available, which is crucial since training these models on CPU would take far too long. The environment check prints out details like the Python version, PyTorch version, GPU name (hopefully a T4 in Colab), CUDA version, and available GPU memory. If no GPU is detected, it shows a warning reminding me to enable it in the runtime settings. This verification step gives confidence that the training will run smoothly before investing time in the actual training process.

In [1]:
# Environment Setup and Package Installation
import sys
import subprocess

def pip_install(packages):
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q"] + packages)

# Core ML stack
pip_install([
    "transformers>=4.44.0",
    "datasets>=2.14.0",
    "accelerate>=0.26.0",
    "evaluate>=0.4.0",
    "openpyxl",
])

# Environment check
try:
    import torch
    import platform
    print("=" * 60)
    print("ENVIRONMENT")
    print("=" * 60)
    print(f"Python: {sys.version.split()[0]} | Platform: {platform.platform()}")
    print(f"PyTorch: {torch.__version__}")
    if torch.cuda.is_available():
        print(f"GPU: {torch.cuda.get_device_name(0)} | CUDA: {torch.version.cuda}")
        print(f"GPU Memory: {round(torch.cuda.get_device_properties(0).total_memory / 1024**3, 2)} GB")
    else:
        print("[WARNING] GPU not detected. Enable T4 GPU in Runtime > Change runtime type.")
    print("=" * 60)
except Exception as e:
    print("Environment check failed:", e)

print("\n[DONE] Setup complete! Ready for training.")

ENVIRONMENT
Python: 3.12.12 | Platform: Linux-6.6.105+-x86_64-with-glibc2.35
PyTorch: 2.8.0+cu126
GPU: Tesla T4 | CUDA: 12.6
GPU Memory: 14.74 GB

[DONE] Setup complete! Ready for training.


## Import Core Libraries and Configure GPU

This cell brings in all the essential libraries that will be used throughout the training process. Starting with torch, which is the foundation for deep learning operations, followed by numpy and pandas for data manipulation. The transformers library imports are particularly important since they provide access to the pre-trained BERT models (AutoTokenizer and AutoModelForQuestionAnswering), along with the Trainer class that handles the training loop. The datasets library is imported to convert pandas DataFrames into a format that works seamlessly with the Trainer.

I included warnings.filterwarnings('ignore') to keep the output clean, though it's worth noting this should be used carefully in production code. The GPU configuration section checks if CUDA is available and sets up the device accordingly. Seeing that GPU check pass is always reassuring since it confirms the notebook will train at a reasonable speed. The cell then handles file upload using Colab's files.upload() function, which opens a dialog to select the cardiovascular dataset CSV. If the upload fails for any reason, it falls back to a default filename, which is a helpful safeguard during development.

In [2]:
# Import Libraries and Upload Dataset
import torch
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import TrainingArguments, Trainer
from transformers import default_data_collator
from datasets import Dataset
import warnings
import time
warnings.filterwarnings('ignore')

# GPU Configuration
print("=" * 60)
print("GPU CONFIGURATION")
print("=" * 60)
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"[OK] GPU Available: {torch.cuda.get_device_name(0)}")
    print(f"CUDA Version: {torch.version.cuda}")
    print(f"GPU Memory: {round(torch.cuda.get_device_properties(0).total_memory / 1024**3, 2)} GB")
else:
    device = torch.device("cpu")
    print("[WARNING] GPU not available; training will be slower.")
print("=" * 60)

# Upload CSV file
print("\nPlease upload your cardiovascular dataset CSV file...")
try:
    from google.colab import files
    uploaded = files.upload()
    CSV_PATH = list(uploaded.keys())[0]
    print(f"[OK] File uploaded: {CSV_PATH}")
except Exception as e:
    print(f"[WARNING] Upload failed: {e}")
    CSV_PATH = "medquadCardiovascular.csv"  # Fallback
    print(f"Using fallback path: {CSV_PATH}")

GPU CONFIGURATION
[OK] GPU Available: Tesla T4
CUDA Version: 12.6
GPU Memory: 14.74 GB

Please upload your cardiovascular dataset CSV file...


Saving medquadCardiovascular.csv to medquadCardiovascular.csv
[OK] File uploaded: medquadCardiovascular.csv


## Load and Prepare the Dataset

This cell loads the cardiovascular question-answer dataset from the uploaded CSV file and prepares it for training. Using `pd.read_csv()`, the data is read into a pandas DataFrame, which makes it easy to explore and manipulate. I print out the total number of records and column names to get a sense of the dataset's structure right away. The sample question and answer length give a quick preview of what kind of data I'm working with, which helps verify that the file loaded correctly.

After the initial load, the cell cleans the data by dropping any rows with missing values in the question or answer columns using `dropna()`. This is important because incomplete samples would cause errors during training. The cleaned dataset is then shuffled randomly with `sample(frac=1.0, random_state=42)` to ensure the model doesn't learn any patterns based on the original ordering. Finally, the data is split into training and validation sets using an 85/15 ratio. The training set will be used to teach the models, while the validation set helps evaluate how well they generalize to unseen questions. Printing the split sizes confirms everything is ready for the next step.

In [3]:
# Load and Prepare Dataset
print("=" * 60)
print("LOADING CARDIOVASCULAR DATASET")
print("=" * 60)

dataset = pd.read_csv(CSV_PATH)
print(f"Total records: {len(dataset)}")
print(f"Columns: {list(dataset.columns)}")
print(f"\nSample question: {str(dataset.iloc[0]['question'])[:80]}...")
print(f"Sample answer length: {len(str(dataset.iloc[0]['answer']))} characters")

# Drop nulls
dataset = dataset.dropna(subset=["question", "answer"]).reset_index(drop=True)
print(f"\nAfter cleaning: {len(dataset)} records")

# Train/val split (85/15)
dataset_shuffled = dataset.sample(frac=1.0, random_state=42).reset_index(drop=True)
split_idx = int(len(dataset_shuffled) * 0.85)
train_data = dataset_shuffled.iloc[:split_idx].copy()
eval_data = dataset_shuffled.iloc[split_idx:].copy()

print(f"\nTrain: {len(train_data)} samples | Validation: {len(eval_data)} samples")
print("=" * 60)

LOADING CARDIOVASCULAR DATASET
Total records: 654
Columns: ['question', 'answer', 'source', 'focus_area']

Sample question: What is (are) High Blood Pressure ?...
Sample answer length: 5586 characters

After cleaning: 654 records

Train: 555 samples | Validation: 99 samples


## Tokenization Function and Metrics Computation

This cell defines two critical functions that will be used throughout the training process. The first function, `prepare_train_features()`, handles tokenization, which is the process of converting text into numerical tokens that BERT models can understand. What makes this function interesting is that it adapts to both cased and uncased models by detecting the tokenizer type. It checks if the model preserves capitalization using `getattr(tokenizer, 'do_lower_case', False)`, which is important because BioBERT is cased while BlueBERT is uncased.

The tokenization process pairs each question with its corresponding answer, truncating longer texts to fit within the MAX_LENGTH of 384 tokens while using a DOC_STRIDE of 128 for overlapping windows. The function then searches for the actual answer position within the tokenized text instead of using fixed positions. For cased models like BioBERT, it performs case-insensitive matching to find where the answer begins, which helps improve accuracy. The answer span length is calculated dynamically based on the answer text length, ranging from 30 to 80 tokens. This adaptive approach should help both models locate answers more accurately.

The second function, `compute_qa_metrics()`, calculates several evaluation metrics after each training epoch. It compares the model's predicted start and end positions with the true positions to compute exact match rates, start accuracy, and end accuracy. More importantly, it calculates F1 score, precision, and recall by treating the answer span as a set of token positions and measuring the overlap between predicted and true tokens. The composite accuracy combines exact match with start and end accuracy, providing a single number that reflects overall performance. These metrics help me understand not just whether the model is getting the exact answer right, but also how close it is when it's wrong.

In [4]:
# Tokenization Function and Dataset Preparation
MAX_LENGTH = 384
DOC_STRIDE = 128

def prepare_train_features(examples, tokenizer):
    # Handle cased vs uncased models by normalizing for search
    is_cased = not getattr(tokenizer, 'do_lower_case', False)

    tokenized = tokenizer(
        examples["question"],
        examples["answer"],
        truncation="only_second",
        max_length=MAX_LENGTH,
        stride=DOC_STRIDE,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    start_positions = []
    end_positions = []

    for i, offsets in enumerate(tokenized["offset_mapping"]):
        sequence_ids = tokenized.sequence_ids(i)

        # Get the answer text
        sample_idx = tokenized["overflow_to_sample_mapping"][i] if "overflow_to_sample_mapping" in tokenized else i
        answer_text = examples["answer"][sample_idx] if isinstance(examples["answer"], list) else examples["answer"]

        # Find context boundaries
        context_start = None
        context_end = None
        for idx, seq_id in enumerate(sequence_ids):
            if seq_id == 1:
                if context_start is None:
                    context_start = idx
                context_end = idx

        if context_start is None or not answer_text:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Extract first meaningful chunk (first 3 words) for better matching
            answer_words = answer_text.strip().split()[:3]
            search_text = " ".join(answer_words)

            # Normalize for case-insensitive matching if model is cased
            if is_cased:
                search_text_lower = search_text.lower()
            else:
                search_text_lower = search_text

            # Find the best token span that matches the answer start
            best_match_idx = context_start
            best_match_score = 0

            for start_idx in range(context_start, min(context_start + 20, context_end)):
                # Decode tokens to find match
                token_span = tokenizer.decode(tokenized["input_ids"][i][start_idx:min(start_idx + 10, context_end)])
                if is_cased:
                    token_span_lower = token_span.lower()
                else:
                    token_span_lower = token_span

                # Calculate match score
                if search_text_lower in token_span_lower:
                    match_score = len(search_text_lower)
                    if match_score > best_match_score:
                        best_match_score = match_score
                        best_match_idx = start_idx

            answer_start = best_match_idx
            # Make answer span reasonable length (30-80 tokens based on answer length)
            span_length = min(max(30, len(answer_text.split()) // 2), 80)
            answer_end = min(answer_start + span_length, context_end)

            start_positions.append(answer_start)
            end_positions.append(answer_end)

    tokenized["start_positions"] = start_positions
    tokenized["end_positions"] = end_positions

    if "offset_mapping" in tokenized:
        tokenized.pop("offset_mapping")

    return tokenized

# Metrics computation function
def compute_qa_metrics(eval_pred):
    predictions, label_ids = eval_pred
    start_logits, end_logits = predictions

    pred_starts = np.argmax(start_logits, axis=1)
    pred_ends = np.argmax(end_logits, axis=1)

    true_starts = np.asarray(label_ids[0]).reshape(-1)
    true_ends = np.asarray(label_ids[1]).reshape(-1)

    exact_match = np.mean((pred_starts == true_starts) & (pred_ends == true_ends))
    start_accuracy = np.mean(pred_starts == true_starts)
    end_accuracy = np.mean(pred_ends == true_ends)

    f1_scores = []
    precision_scores = []
    recall_scores = []

    for ps, pe, ts, te in zip(pred_starts, pred_ends, true_starts, true_ends):
        ps, pe, ts, te = int(ps), int(pe), int(ts), int(te)
        pred_tokens = set(range(ps, pe + 1))
        true_tokens = set(range(ts, te + 1))

        if not pred_tokens and not true_tokens:
            f1_scores.append(1.0)
            precision_scores.append(1.0)
            recall_scores.append(1.0)
        elif not pred_tokens or not true_tokens:
            f1_scores.append(0.0)
            precision_scores.append(0.0)
            recall_scores.append(0.0)
        else:
            common = len(pred_tokens & true_tokens)
            if common == 0:
                f1_scores.append(0.0)
                precision_scores.append(0.0)
                recall_scores.append(0.0)
            else:
                precision = common / len(pred_tokens)
                recall = common / len(true_tokens)
                precision_scores.append(precision)
                recall_scores.append(recall)
                f1_scores.append(2 * (precision * recall) / (precision + recall))

    accuracy = (exact_match + start_accuracy + end_accuracy) / 3

    return {
        "accuracy": float(accuracy),
        "exact_match": float(exact_match),
        "start_accuracy": float(start_accuracy),
        "end_accuracy": float(end_accuracy),
        "f1": float(np.mean(f1_scores)) if f1_scores else 0.0,
        "precision": float(np.mean(precision_scores)) if precision_scores else 0.0,
        "recall": float(np.mean(recall_scores)) if recall_scores else 0.0,
    }

print("[DONE] Tokenization and metrics functions ready.")

[DONE] Tokenization and metrics functions ready.


## BlueBERT Training Loop - 10 Iterations

This cell executes the complete training process for BlueBERT across 10 different hyperparameter configurations. The model being used is "aaditya/Bluebert_emrqa", which is an uncased BERT model that has been pre-trained on clinical text and fine-tuned for question answering tasks. I chose to test 10 different configurations because finding the optimal hyperparameters requires experimentation. Each configuration varies the number of epochs, learning rate, batch size, gradient accumulation steps, warmup ratio, and weight decay to explore how these parameters affect performance.

The training process starts by loading the tokenizer once and applying it to both the training and validation datasets using the `prepare_train_features()` function defined earlier. This tokenization step happens before the training loop to avoid repeating it unnecessarily. Then, for each of the 10 iterations, a fresh model is loaded from the pre-trained checkpoint to ensure each configuration starts from the same baseline. This is important for fair comparison.

For each iteration, the cell sets up TrainingArguments with the specific hyperparameters for that run. Notable settings include fp16=True for mixed precision training, which speeds things up and reduces memory usage on the T4 GPU, and eval_strategy="epoch" so the model is evaluated after each epoch. The Trainer class handles the actual training loop, computing gradients, updating weights, and calling the metrics function. After training completes, the cell evaluates the model on the validation set and records all the metrics along with the runtime.

At the end of each iteration, the model and trainer are explicitly deleted and the GPU cache is cleared using `torch.cuda.empty_cache()`. This prevents memory from accumulating across iterations, which could cause out-of-memory errors. Finally, after all 10 iterations complete, the results are displayed in a summary table showing which configuration performed best. This systematic approach makes it easier to identify the optimal hyperparameters for BlueBERT on this cardiovascular dataset.

In [5]:
# BlueBERT Training - Iterations 1-10
print("\n" + "=" * 60)
print("BLUEBERT TRAINING - ITERATIONS 1-10")
print("=" * 60)

MODEL_NAME_BLUEBERT = "aaditya/Bluebert_emrqa"
print(f"Model: {MODEL_NAME_BLUEBERT}\n")

# Hyperparameter configurations (optimized for T4 + 650 rows)
hyperparam_sets_bluebert = [
    {"iter": 1,  "epochs": 5, "lr": 3e-5, "batch": 16, "grad_accum": 2, "warmup": 0.1,  "wd": 0.01},
    {"iter": 2,  "epochs": 6, "lr": 3e-5, "batch": 16, "grad_accum": 2, "warmup": 0.1,  "wd": 0.01},
    {"iter": 3,  "epochs": 5, "lr": 4e-5, "batch": 16, "grad_accum": 2, "warmup": 0.1,  "wd": 0.01},
    {"iter": 4,  "epochs": 6, "lr": 4e-5, "batch": 16, "grad_accum": 2, "warmup": 0.1,  "wd": 0.01},
    {"iter": 5,  "epochs": 5, "lr": 5e-5, "batch": 16, "grad_accum": 2, "warmup": 0.1,  "wd": 0.01},
    {"iter": 6,  "epochs": 8, "lr": 2e-5, "batch": 16, "grad_accum": 2, "warmup": 0.15, "wd": 0.01},
    {"iter": 7,  "epochs": 8, "lr": 3e-5, "batch": 16, "grad_accum": 2, "warmup": 0.15, "wd": 0.01},
    {"iter": 8,  "epochs": 6, "lr": 3e-5, "batch": 8,  "grad_accum": 4, "warmup": 0.1,  "wd": 0.01},
    {"iter": 9,  "epochs": 6, "lr": 4e-5, "batch": 8,  "grad_accum": 4, "warmup": 0.15, "wd": 0.01},
    {"iter": 10, "epochs": 5, "lr": 5e-5, "batch": 8,  "grad_accum": 4, "warmup": 0.05, "wd": 0.02},
]

# Load tokenizer (once)
tokenizer_bluebert = AutoTokenizer.from_pretrained(MODEL_NAME_BLUEBERT, use_fast=True)

# Tokenize datasets (once)
print("Tokenizing datasets...")
train_ds = Dataset.from_pandas(train_data)
eval_ds = Dataset.from_pandas(eval_data)

tokenized_train_bluebert = train_ds.map(
    lambda x: prepare_train_features(x, tokenizer_bluebert),
    batched=True,
    remove_columns=train_ds.column_names,
    desc="Tokenizing train",
)

tokenized_eval_bluebert = eval_ds.map(
    lambda x: prepare_train_features(x, tokenizer_bluebert),
    batched=True,
    remove_columns=eval_ds.column_names,
    desc="Tokenizing eval",
)
print("[DONE] Tokenization complete.\n")

# Storage for results
results_bluebert = []

# Training loop
for params in hyperparam_sets_bluebert:
    print(f"\n{'='*60}")
    print(f"BlueBERT Iteration {params['iter']}/10")
    print(f"Config: {params['epochs']} epochs, LR={params['lr']}, Batch={params['batch']}, Grad Accum={params['grad_accum']}")
    print(f"{'='*60}")

    # Load fresh model
    model_bluebert = AutoModelForQuestionAnswering.from_pretrained(MODEL_NAME_BLUEBERT)
    model_bluebert.to(device)

    # Training arguments
    training_args = TrainingArguments(
        output_dir=f"./bluebert_iter_{params['iter']}",
        num_train_epochs=params["epochs"],
        per_device_train_batch_size=params["batch"],
        per_device_eval_batch_size=params["batch"],
        gradient_accumulation_steps=params["grad_accum"],
        learning_rate=params["lr"],
        warmup_ratio=params["warmup"],
        weight_decay=params["wd"],
        fp16=torch.cuda.is_available(),
        dataloader_pin_memory=True,
        dataloader_num_workers=2,
        eval_strategy="epoch",
        save_strategy="no",
        logging_steps=50,
        report_to=[],
        disable_tqdm=False,
        seed=42,
    )

    # Initialize trainer
    trainer = Trainer(
        model=model_bluebert,
        args=training_args,
        train_dataset=tokenized_train_bluebert,
        eval_dataset=tokenized_eval_bluebert,
        tokenizer=tokenizer_bluebert,
        data_collator=default_data_collator,
        compute_metrics=compute_qa_metrics,
    )

    # Train and measure time
    start_time = time.time()
    trainer.train()
    runtime = round(time.time() - start_time, 2)

    # Evaluate
    eval_results = trainer.evaluate()

    # Extract metrics
    accuracy = eval_results.get("eval_accuracy", 0)
    f1 = eval_results.get("eval_f1", 0)
    precision = eval_results.get("eval_precision", 0)
    recall = eval_results.get("eval_recall", 0)

    # Store results
    results_bluebert.append({
        "Iteration": params["iter"],
        "Epochs": params["epochs"],
        "Learning Rate": params["lr"],
        "Batch Size": params["batch"],
        "Grad Accum": params["grad_accum"],
        "Warmup": params["warmup"],
        "Weight Decay": params["wd"],
        "Accuracy": round(accuracy * 100, 2),
        "F1 Score": round(f1 * 100, 2),
        "Precision": round(precision * 100, 2),
        "Recall": round(recall * 100, 2),
        "Runtime (s)": runtime,
    })

    print(f"\n[DONE] Iteration {params['iter']} complete")
    print(f"Accuracy: {accuracy*100:.2f}%, F1: {f1*100:.2f}%, Precision: {precision*100:.2f}%, Recall: {recall*100:.2f}%")
    print(f"Runtime: {runtime}s")

    # Clean up
    del model_bluebert, trainer
    torch.cuda.empty_cache()

# Summary
print(f"\n{'='*60}")
print("BLUEBERT TRAINING COMPLETE")
print(f"{'='*60}")
df_bluebert = pd.DataFrame(results_bluebert)
print(df_bluebert.to_string(index=False))
print(f"\nTotal runtime: {sum([r['Runtime (s)'] for r in results_bluebert]):.2f}s")



BLUEBERT TRAINING - ITERATIONS 1-10
Model: aaditya/Bluebert_emrqa



tokenizer_config.json:   0%|          | 0.00/402 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

...


[DONE] Iteration 8 complete
Accuracy: 46.80%, F1: 97.50%, Precision: 97.06%, Recall: 98.44%
Runtime: 128.38s

BlueBERT Iteration 9/10
Config: 6 epochs, LR=4e-05, Batch=8, Grad Accum=4


Epoch,Training Loss,Validation Loss,Accuracy,Exact Match,Start Accuracy,End Accuracy,F1,Precision,Recall
1,No log,1.949353,0.402357,0.106061,0.994949,0.106061,0.945916,0.939350,0.967774
2,5.987400,1.301660,0.424242,0.136364,1.000000,0.136364,0.968672,0.960678,0.983636
3,5.987400,1.203000,0.468013,0.202020,1.000000,0.202020,0.974182,0.975568,0.978391
4,1.275500,1.220100,0.484848,0.227273,1.000000,0.227273,0.979760,0.971801,0.990778
5,0.917600,1.188803,0.505051,0.257576,1.000000,0.257576,0.979598,0.975215,0.987409
6,0.917600,1.214664,0.535354,0.303030,1.000000,0.303030,0.981316,0.979191,0.986240



[DONE] Iteration 9 complete
Accuracy: 53.54%, F1: 98.13%, Precision: 97.92%, Recall: 98.62%
Runtime: 128.96s

BlueBERT Iteration 10/10
Config: 5 epochs, LR=5e-05, Batch=8, Grad Accum=4


Epoch,Training Loss,Validation Loss,Accuracy,Exact Match,Start Accuracy,End Accuracy,F1,Precision,Recall
1,No log,1.439033,0.424242,0.136364,1.000000,0.136364,0.959374,0.952869,0.974772
2,4.042000,1.225405,0.444444,0.166667,1.000000,0.166667,0.973924,0.969892,0.982858
3,4.042000,1.182674,0.451178,0.176768,1.000000,0.176768,0.979437,0.980421,0.981764
4,1.184500,1.197041,0.457912,0.186869,1.000000,0.186869,0.981558,0.977348,0.988029
5,0.859100,1.170073,0.494949,0.242424,1.000000,0.242424,0.982819,0.982273,0.985423



[DONE] Iteration 10 complete
Accuracy: 49.49%, F1: 98.28%, Precision: 98.23%, Recall: 98.54%
Runtime: 107.44s

BLUEBERT TRAINING COMPLETE
 Iteration  Epochs  Learning Rate  Batch Size  Grad Accum  Warmup  Weight Decay  Accuracy  F1 Score  Precision  Recall  Runtime (s)
         1       5        0.00003          16           2    0.10          0.01     46.13     97.57      97.24   98.34       107.33
         2       6        0.00003          16           2    0.10          0.01     47.81     97.64      97.72   97.99       124.86
         3       5        0.00004          16           2    0.10          0.01     48.82     97.93      97.73   98.49       104.07
         4       6        0.00004          16           2    0.10          0.01     60.94     98.21      98.03   98.71       124.30
         5       5        0.00005          16           2    0.10          0.01     60.61     98.42      98.07   99.02       104.31
         6       8        0.00002          16           2    0.15    

## BioBERT Training Loop - 10 Iterations

This cell mirrors the BlueBERT training process but uses the BioBERT model instead. The model identifier is "dmis-lab/biobert-base-cased-v1.1", which is a cased BERT variant that was pre-trained on biomedical literature from PubMed. The key difference between this and BlueBERT is that BioBERT preserves capitalization in the text, which can be important for medical terminology where case sometimes carries meaning. To ensure a fair comparison, this cell uses exactly the same hyperparameter configurations as the BlueBERT training.

The structure follows the same pattern as before. The BioBERT tokenizer is loaded and applied to the training and validation datasets. Since the tokenization function I wrote earlier detects whether a model is cased or uncased, it automatically adjusts the answer-finding logic for BioBERT's cased tokenizer. This should help BioBERT achieve better exact match scores compared to using fixed answer positions.

Each of the 10 iterations loads a fresh BioBERT model, trains it with the specified hyperparameters, evaluates it on the validation set, and records the performance metrics. The training arguments are identical to those used for BlueBERT, including mixed precision training, gradient accumulation, and warmup steps. After each iteration, memory is carefully cleaned up to prevent GPU memory issues.

Once all 10 iterations complete, the results are displayed in a summary table. By comparing the BioBERT results with the BlueBERT results from the previous cell, I can determine which model architecture performs better for cardiovascular question answering and which hyperparameter configuration works best for each model. This side-by-side comparison is the core of the experiment.

In [6]:
# BioBERT Training - Iterations 1-10
print("\n" + "=" * 60)
print("BIOBERT TRAINING - ITERATIONS 1-10")
print("=" * 60)

MODEL_NAME_BIOBERT = "dmis-lab/biobert-base-cased-v1.1"
print(f"Model: {MODEL_NAME_BIOBERT}\n")

# Same hyperparameter configurations
hyperparam_sets_biobert = [
    {"iter": 1,  "epochs": 5, "lr": 3e-5, "batch": 16, "grad_accum": 2, "warmup": 0.1,  "wd": 0.01},
    {"iter": 2,  "epochs": 6, "lr": 3e-5, "batch": 16, "grad_accum": 2, "warmup": 0.1,  "wd": 0.01},
    {"iter": 3,  "epochs": 5, "lr": 4e-5, "batch": 16, "grad_accum": 2, "warmup": 0.1,  "wd": 0.01},
    {"iter": 4,  "epochs": 6, "lr": 4e-5, "batch": 16, "grad_accum": 2, "warmup": 0.1,  "wd": 0.01},
    {"iter": 5,  "epochs": 5, "lr": 5e-5, "batch": 16, "grad_accum": 2, "warmup": 0.1,  "wd": 0.01},
    {"iter": 6,  "epochs": 8, "lr": 2e-5, "batch": 16, "grad_accum": 2, "warmup": 0.15, "wd": 0.01},
    {"iter": 7,  "epochs": 8, "lr": 3e-5, "batch": 16, "grad_accum": 2, "warmup": 0.15, "wd": 0.01},
    {"iter": 8,  "epochs": 6, "lr": 3e-5, "batch": 8,  "grad_accum": 4, "warmup": 0.1,  "wd": 0.01},
    {"iter": 9,  "epochs": 6, "lr": 4e-5, "batch": 8,  "grad_accum": 4, "warmup": 0.15, "wd": 0.01},
    {"iter": 10, "epochs": 5, "lr": 5e-5, "batch": 8,  "grad_accum": 4, "warmup": 0.05, "wd": 0.02},
]

# Load tokenizer (once)
tokenizer_biobert = AutoTokenizer.from_pretrained(MODEL_NAME_BIOBERT, use_fast=True)

# Tokenize datasets (once)
print("Tokenizing datasets...")
tokenized_train_biobert = train_ds.map(
    lambda x: prepare_train_features(x, tokenizer_biobert),
    batched=True,
    remove_columns=train_ds.column_names,
    desc="Tokenizing train",
)

tokenized_eval_biobert = eval_ds.map(
    lambda x: prepare_train_features(x, tokenizer_biobert),
    batched=True,
    remove_columns=eval_ds.column_names,
    desc="Tokenizing eval",
)
print("[DONE] Tokenization complete.\n")

# Storage for results
results_biobert = []

# Training loop
for params in hyperparam_sets_biobert:
    print(f"\n{'='*60}")
    print(f"BioBERT Iteration {params['iter']}/10")
    print(f"Config: {params['epochs']} epochs, LR={params['lr']}, Batch={params['batch']}, Grad Accum={params['grad_accum']}")
    print(f"{'='*60}")

    # Load fresh model
    model_biobert = AutoModelForQuestionAnswering.from_pretrained(MODEL_NAME_BIOBERT)
    model_biobert.to(device)

    # Training arguments
    training_args = TrainingArguments(
        output_dir=f"./biobert_iter_{params['iter']}",
        num_train_epochs=params["epochs"],
        per_device_train_batch_size=params["batch"],
        per_device_eval_batch_size=params["batch"],
        gradient_accumulation_steps=params["grad_accum"],
        learning_rate=params["lr"],
        warmup_ratio=params["warmup"],
        weight_decay=params["wd"],
        fp16=torch.cuda.is_available(),
        dataloader_pin_memory=True,
        dataloader_num_workers=2,
        eval_strategy="epoch",
        save_strategy="no",
        logging_steps=50,
        report_to=[],
        disable_tqdm=False,
        seed=42,
    )

    # Initialize trainer
    trainer = Trainer(
        model=model_biobert,
        args=training_args,
        train_dataset=tokenized_train_biobert,
        eval_dataset=tokenized_eval_biobert,
        tokenizer=tokenizer_biobert,
        data_collator=default_data_collator,
        compute_metrics=compute_qa_metrics,
    )

    # Train and measure time
    start_time = time.time()
    trainer.train()
    runtime = round(time.time() - start_time, 2)

    # Evaluate
    eval_results = trainer.evaluate()

    # Extract metrics
    accuracy = eval_results.get("eval_accuracy", 0)
    f1 = eval_results.get("eval_f1", 0)
    precision = eval_results.get("eval_precision", 0)
    recall = eval_results.get("eval_recall", 0)

    # Store results
    results_biobert.append({
        "Iteration": params["iter"],
        "Epochs": params["epochs"],
        "Learning Rate": params["lr"],
        "Batch Size": params["batch"],
        "Grad Accum": params["grad_accum"],
        "Warmup": params["warmup"],
        "Weight Decay": params["wd"],
        "Accuracy": round(accuracy * 100, 2),
        "F1 Score": round(f1 * 100, 2),
        "Precision": round(precision * 100, 2),
        "Recall": round(recall * 100, 2),
        "Runtime (s)": runtime,
    })

    print(f"\n[DONE] Iteration {params['iter']} complete")
    print(f"Accuracy: {accuracy*100:.2f}%, F1: {f1*100:.2f}%, Precision: {precision*100:.2f}%, Recall: {recall*100:.2f}%")
    print(f"Runtime: {runtime}s")

    # Clean up
    del model_biobert, trainer
    torch.cuda.empty_cache()

# Summary
print(f"\n{'='*60}")
print("BIOBERT TRAINING COMPLETE")
print(f"{'='*60}")
df_biobert = pd.DataFrame(results_biobert)
print(df_biobert.to_string(index=False))
print(f"\nTotal runtime: {sum([r['Runtime (s)'] for r in results_biobert]):.2f}s")


BIOBERT TRAINING - ITERATIONS 1-10
Model: dmis-lab/biobert-base-cased-v1.1



config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Tokenizing datasets...


Tokenizing train:   0%|          | 0/555 [00:00<?, ? examples/s]

...

Epoch,Training Loss,Validation Loss,Accuracy,Exact Match,Start Accuracy,End Accuracy,F1,Precision,Recall
1,No log,1.762414,0.401294,0.106796,0.990291,0.106796,0.888869,0.905760,0.890417
2,3.737300,1.450807,0.423948,0.135922,1.000000,0.135922,0.948785,0.944551,0.962122
3,1.442900,1.394487,0.436893,0.155340,1.000000,0.155340,0.965520,0.953220,0.981652
4,1.442900,1.358894,0.446602,0.169903,1.000000,0.169903,0.978028,0.968489,0.990203
5,1.091800,1.235798,0.453074,0.179612,1.000000,0.179612,0.972851,0.978545,0.973773
6,0.864700,1.245851,0.472492,0.208738,1.000000,0.208738,0.974441,0.978744,0.976416



[DONE] Iteration 9 complete
Accuracy: 47.25%, F1: 97.44%, Precision: 97.87%, Recall: 97.64%
Runtime: 133.37s

BioBERT Iteration 10/10
Config: 5 epochs, LR=5e-05, Batch=8, Grad Accum=4


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.1 and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Exact Match,Start Accuracy,End Accuracy,F1,Precision,Recall
1,No log,1.486608,0.443366,0.165049,1.000000,0.165049,0.938616,0.933991,0.950800
2,2.829300,1.406991,0.414239,0.121359,1.000000,0.121359,0.958689,0.944014,0.977299
3,1.273200,1.015825,0.517799,0.276699,1.000000,0.276699,0.968019,0.961975,0.976816
4,1.273200,1.204675,0.446602,0.169903,1.000000,0.169903,0.974198,0.971774,0.982760
5,0.917000,0.928929,0.592233,0.388350,1.000000,0.388350,0.963754,0.972209,0.965537



[DONE] Iteration 10 complete
Accuracy: 59.22%, F1: 96.38%, Precision: 97.22%, Recall: 96.55%
Runtime: 110.97s

BIOBERT TRAINING COMPLETE
 Iteration  Epochs  Learning Rate  Batch Size  Grad Accum  Warmup  Weight Decay  Accuracy  F1 Score  Precision  Recall  Runtime (s)
         1       5        0.00003          16           2    0.10          0.01     42.39     96.33      95.65   97.29       108.47
         2       6        0.00003          16           2    0.10          0.01     42.72     97.08      96.58   97.95       129.05
         3       5        0.00004          16           2    0.10          0.01     44.66     97.18      97.44   97.56       107.98
         4       6        0.00004          16           2    0.10          0.01     47.57     97.84      97.70   98.18       128.86
         5       5        0.00005          16           2    0.10          0.01     44.66     97.25      97.56   97.60       107.35
         6       8        0.00002          16           2    0.15     

## Compare Results and Identify Best Models

This cell analyzes the results from both training loops to determine which model performed best. After running 10 iterations for each model, I need to identify the single best configuration for BlueBERT and the single best configuration for BioBERT. The comparison is based on F1 score, which is the most reliable metric for question answering tasks since it measures how well the model captures the right answer tokens, even if it doesn't match exactly.

Using Python's `max()` function with a lambda key, the cell finds the iteration with the highest F1 score for each model. It then prints out detailed information about both winners, including not just the F1 score but also accuracy, precision, recall, and the specific hyperparameters that produced those results. This information is valuable because it tells me which combination of epochs, learning rate, batch size, and other settings worked best for each model architecture.

The cell then creates a side-by-side comparison DataFrame to make it easier to see the differences at a glance. Finally, it declares an overall winner by comparing the F1 scores of the two best models. While both models might perform well, this helps establish which one is ultimately better suited for answering cardiovascular health questions. The results from this analysis will guide the next steps, where the best models are retrained and saved for future use.

In [7]:
# Compare Results and Select Best Models
print("\n" + "=" * 60)
print("MODEL COMPARISON")
print("=" * 60)

# Find best iteration for each model (based on F1 score)
best_bluebert = max(results_bluebert, key=lambda x: x["F1 Score"])
best_biobert = max(results_biobert, key=lambda x: x["F1 Score"])

print("\nBLUEBERT Best Model:")
print(f"  Iteration: {best_bluebert['Iteration']}")
print(f"  F1 Score: {best_bluebert['F1 Score']}%")
print(f"  Accuracy: {best_bluebert['Accuracy']}%")
print(f"  Precision: {best_bluebert['Precision']}%")
print(f"  Recall: {best_bluebert['Recall']}%")
print(f"  Config: {best_bluebert['Epochs']} epochs, LR={best_bluebert['Learning Rate']}")
print(f"  Runtime: {best_bluebert['Runtime (s)']}s")

print("\nBIOBERT Best Model:")
print(f"  Iteration: {best_biobert['Iteration']}")
print(f"  F1 Score: {best_biobert['F1 Score']}%")
print(f"  Accuracy: {best_biobert['Accuracy']}%")
print(f"  Precision: {best_biobert['Precision']}%")
print(f"  Recall: {best_biobert['Recall']}%")
print(f"  Config: {best_biobert['Epochs']} epochs, LR={best_biobert['Learning Rate']}")
print(f"  Runtime: {best_biobert['Runtime (s)']}s")

# Side-by-side comparison
print("\n" + "=" * 60)
print("BEST MODELS COMPARISON")
print("=" * 60)
comparison_df = pd.DataFrame([
    {"Model": "BlueBERT", "Iteration": best_bluebert["Iteration"],
     "F1": best_bluebert["F1 Score"], "Accuracy": best_bluebert["Accuracy"],
     "Precision": best_bluebert["Precision"], "Recall": best_bluebert["Recall"],
     "Runtime (s)": best_bluebert["Runtime (s)"]},
    {"Model": "BioBERT", "Iteration": best_biobert["Iteration"],
     "F1": best_biobert["F1 Score"], "Accuracy": best_biobert["Accuracy"],
     "Precision": best_biobert["Precision"], "Recall": best_biobert["Recall"],
     "Runtime (s)": best_biobert["Runtime (s)"]}
])
print(comparison_df.to_string(index=False))

# Determine overall winner
winner = "BlueBERT" if best_bluebert["F1 Score"] > best_biobert["F1 Score"] else "BioBERT"
print(f"\n[WINNER] Overall Winner: {winner}")


MODEL COMPARISON

BLUEBERT Best Model:
  Iteration: 5
  F1 Score: 98.42%
  Accuracy: 60.61%
  Precision: 98.07%
  Recall: 99.02%
  Config: 5 epochs, LR=5e-05
  Runtime: 104.31s

BIOBERT Best Model:
  Iteration: 4
  F1 Score: 97.84%
  Accuracy: 47.57%
  Precision: 97.7%
  Recall: 98.18%
  Config: 6 epochs, LR=4e-05
  Runtime: 128.86s

BEST MODELS COMPARISON
   Model  Iteration    F1  Accuracy  Precision  Recall  Runtime (s)
BlueBERT          5 98.42     60.61      98.07   99.02       104.31
 BioBERT          4 97.84     47.57      97.70   98.18       128.86

[WINNER] Overall Winner: BlueBERT


## Retrain and Save Best Models

This cell takes the best-performing configurations identified in the previous step and retrains both models from scratch, this time saving them to disk. While the earlier training loop evaluated many configurations and discarded the models to save memory, this cell focuses on preserving the winners for future use. Having these trained models saved means I can load them later for inference without needing to retrain from scratch.

For BlueBERT, the cell retrieves the hyperparameters from the best iteration and sets up a new training run with those exact settings. The key difference this time is that `save_strategy="epoch"` is enabled, allowing the Trainer to save checkpoints during training. After training completes, `save_model()` writes the model weights to the "./bluebert_best_model" directory, and `save_pretrained()` saves the tokenizer configuration alongside it. This ensures that everything needed to use the model later is packaged together.

The same process repeats for BioBERT, retraining it with its best hyperparameters and saving it to "./biobert_best_model". Retraining might seem redundant since these models were already trained once, but it's necessary because the earlier models were deleted to free up memory. By the end of this cell, I have two fully trained models saved on disk, ready to be loaded and used for answering cardiovascular health questions. These saved models represent the culmination of all the hyperparameter tuning work.

In [8]:
# Save Best Models
print("\n" + "=" * 60)
print("SAVING BEST MODELS")
print("=" * 60)

# Retrain and save best BlueBERT model
print("\nRetraining best BlueBERT model...")
best_bluebert_config = hyperparam_sets_bluebert[best_bluebert["Iteration"] - 1]

model_bluebert_best = AutoModelForQuestionAnswering.from_pretrained(MODEL_NAME_BLUEBERT)
model_bluebert_best.to(device)

training_args_best_bluebert = TrainingArguments(
    output_dir="./bluebert_best_model",
    num_train_epochs=best_bluebert_config["epochs"],
    per_device_train_batch_size=best_bluebert_config["batch"],
    per_device_eval_batch_size=best_bluebert_config["batch"],
    gradient_accumulation_steps=best_bluebert_config["grad_accum"],
    learning_rate=best_bluebert_config["lr"],
    warmup_ratio=best_bluebert_config["warmup"],
    weight_decay=best_bluebert_config["wd"],
    fp16=torch.cuda.is_available(),
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    logging_steps=50,
    report_to=[],
    disable_tqdm=False,
    seed=42,
)

trainer_best_bluebert = Trainer(
    model=model_bluebert_best,
    args=training_args_best_bluebert,
    train_dataset=tokenized_train_bluebert,
    eval_dataset=tokenized_eval_bluebert,
    tokenizer=tokenizer_bluebert,
    data_collator=default_data_collator,
    compute_metrics=compute_qa_metrics,
)

trainer_best_bluebert.train()
trainer_best_bluebert.save_model("./bluebert_best_model")
tokenizer_bluebert.save_pretrained("./bluebert_best_model")
print("[DONE] BlueBERT best model saved to ./bluebert_best_model")

# Retrain and save best BioBERT model
print("\nRetraining best BioBERT model...")
best_biobert_config = hyperparam_sets_biobert[best_biobert["Iteration"] - 1]

model_biobert_best = AutoModelForQuestionAnswering.from_pretrained(MODEL_NAME_BIOBERT)
model_biobert_best.to(device)

training_args_best_biobert = TrainingArguments(
    output_dir="./biobert_best_model",
    num_train_epochs=best_biobert_config["epochs"],
    per_device_train_batch_size=best_biobert_config["batch"],
    per_device_eval_batch_size=best_biobert_config["batch"],
    gradient_accumulation_steps=best_biobert_config["grad_accum"],
    learning_rate=best_biobert_config["lr"],
    warmup_ratio=best_biobert_config["warmup"],
    weight_decay=best_biobert_config["wd"],
    fp16=torch.cuda.is_available(),
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    logging_steps=50,
    report_to=[],
    disable_tqdm=False,
    seed=42,
)

trainer_best_biobert = Trainer(
    model=model_biobert_best,
    args=training_args_best_biobert,
    train_dataset=tokenized_train_biobert,
    eval_dataset=tokenized_eval_biobert,
    tokenizer=tokenizer_biobert,
    data_collator=default_data_collator,
    compute_metrics=compute_qa_metrics,
)

trainer_best_biobert.train()
trainer_best_biobert.save_model("./biobert_best_model")
tokenizer_biobert.save_pretrained("./biobert_best_model")
print("[DONE] BioBERT best model saved to ./biobert_best_model")

print("\n[DONE] Both best models saved successfully!")


SAVING BEST MODELS

Retraining best BlueBERT model...


Epoch,Training Loss,Validation Loss,Accuracy,Exact Match,Start Accuracy,End Accuracy,F1,Precision,Recall
1,No log,1.479714,0.437710,0.156566,1.000000,0.156566,0.961071,0.956018,0.974136
2,4.687900,1.219489,0.447811,0.171717,1.000000,0.171717,0.975115,0.970818,0.983917
3,4.687900,1.169102,0.505051,0.257576,1.000000,0.257576,0.978752,0.980454,0.980912
4,1.186700,1.059905,0.572391,0.358586,1.000000,0.358586,0.982806,0.974293,0.993908
5,0.780900,1.096428,0.612795,0.419192,1.000000,0.419192,0.984896,0.980332,0.991690


[DONE] BlueBERT best model saved to ./bluebert_best_model

Retraining best BioBERT model...


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.1 and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Exact Match,Start Accuracy,End Accuracy,F1,Precision,Recall
1,No log,1.659069,0.396440,0.097087,0.995146,0.097087,0.922007,0.941060,0.925016
2,3.520300,1.382388,0.433657,0.150485,1.000000,0.150485,0.959779,0.955460,0.969806
3,1.383500,1.370648,0.394822,0.092233,1.000000,0.092233,0.953064,0.950427,0.964761
4,1.383500,1.536507,0.427184,0.140777,1.000000,0.140777,0.966533,0.969075,0.975902
5,1.109000,1.398014,0.436893,0.155340,1.000000,0.155340,0.969069,0.976201,0.973134
6,0.942300,1.492897,0.423948,0.135922,1.000000,0.135922,0.969552,0.980301,0.969874


[DONE] BioBERT best model saved to ./biobert_best_model

[DONE] Both best models saved successfully!
